In [10]:
import numpy as np
import os
import pandas as pd
import pathlib
import itertools
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
from efficientnet_pytorch import EfficientNet

# project_dir = pathlib.Path(__file__).parent.absolute()

# paths to data dirs
# lc_train_path = "/home/dsvm113/IdeaProjects/workspace/data_1/training_set/noisy_train/"
# params_train_path = "/home/dsvm113/IdeaProjects/workspace/data_1/training_set/params_train/"
lc_train_path = "./data/noisy_train/"
params_train_path = "./data/params_train/"

In [12]:
tfms = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
img = tfms(Image.open('data/0001_01.png')).unsqueeze(0)
print(img.shape) # torch.Size([1, 3, 224, 224])

torch.Size([1, 3, 224, 224])


In [13]:
model = EfficientNet.from_pretrained("efficientnet-b0")

features = model.extract_features(img)
print(features.shape)

Loaded pretrained weights for efficientnet-b0
torch.Size([1, 1280, 7, 7])


In [2]:
item_lc_path = lc_train_path + "0100_01_01.txt"

In [3]:
test = np.loadtxt(item_lc_path)
test = torch.from_numpy(test)

In [11]:
test.size()

torch.Size([55, 300])

In [13]:
some = torch.split(test, 55)
some

(tensor([[0.5048, 0.7209, 0.8134,  ..., 1.0003, 0.9997, 0.9991],
         [0.5039, 0.7216, 0.8138,  ..., 0.9991, 1.0003, 1.0003],
         [0.5042, 0.7222, 0.8142,  ..., 1.0002, 1.0004, 0.9994],
         ...,
         [0.5041, 0.7236, 0.8173,  ..., 0.9990, 0.9977, 0.9982],
         [0.5043, 0.7202, 0.8123,  ..., 0.9975, 1.0022, 1.0007],
         [0.5037, 0.7238, 0.8101,  ..., 1.0021, 1.0060, 1.0080]],
        dtype=torch.float64),)

In [19]:
some[0][0].shape

torch.Size([300])

In [6]:
with open(item_lc_path, "r") as f:
    temp_storage_str = list(itertools.islice(f, 6))

temp_storage_float = []

for string in temp_storage_str:
    # Separate the digits and the non-digits.
    new_str = ["".join(x) for _, x in itertools.groupby(string, key=str.isdigit)]

    # Only new_str[0] is the one we want to omit.
    # We want to join back into a single string because "." previously is classifed
    # as non-digit. 
    new_str = "".join(new_str[1:])  

    # Convert to float. 
    temp_storage_float.append(float(new_str))

In [7]:
temp_storage_float

[4880.0, 4.5, 0.73, 0.77, 9.688, 6.67202]

In [8]:
temp_storage_float.pop(-5)
temp_storage_float

[4880.0, 0.73, 0.77, 9.688, 6.67202]

In [9]:
if temp_storage_float[-4] > 0.5:
    temp_storage_float[-4] = 0

temp_storage_float

[4880.0, 0, 0.77, 9.688, 6.67202]

In [10]:
temp_numpy = np.array(temp_storage_float)
temp_numpy

array([4.88000e+03, 0.00000e+00, 7.70000e-01, 9.68800e+00, 6.67202e+00])

In [11]:
np.where(temp_numpy[-2] > 10, 0, temp_numpy)

array([4.88000e+03, 0.00000e+00, 7.70000e-01, 9.68800e+00, 6.67202e+00])

In [12]:
# General imports
import numpy as np
import matplotlib.pylab as plt
import torch
    
from pathlib import Path

from utils import BaselineConv

baseline = BaselineConv().double().to("cpu")
baseline

BaselineConv(
  (layer1): Conv1d(16505, 1024, kernel_size=(1,), stride=(2,))
  (act1): ReLU()
  (layer2): Conv1d(1024, 256, kernel_size=(1,), stride=(1,))
  (act2): ReLU()
  (layer3): Linear(in_features=256, out_features=55, bias=True)
)